# AskBERT

Using [AskBERT](https://github.com/rajammanabrolu/WorldGeneration/tree/master/neural-based/KG-extraction), [BERT-SQuAD](https://github.com/kamalkraj/BERT-SQuAD). 

Paper: Ammanabrolu, Prithviraj, Wesley Cheung, Dan Tu, William Broniec, and Mark Riedl. 2020. “Bringing Stories Alive: Generating Interactive Fiction Worlds.” Proceedings of the AAAI Conference on Artificial Intelligence and Interactive Digital Entertainment 16 (1): 3–9. https://doi.org/10.1609/aiide.v16i1.7400.


In [ ]:
#%pip install pytorch-transformers>=1.0.0 Flask>=1.1.1 Flask-Cors>=3.0.8

## Inference on BERT-SQuAD

In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

In [ ]:
from bert import QA

model = QA('model')

doc = "Victoria has a written constitution enacted in 1975, but based on the 1855 colonial constitution, passed by the United Kingdom Parliament as the Victoria Constitution Act 1855, which establishes the Parliament as the state's law-making body for matters coming under state responsibility. The Victorian Constitution can be amended by the Parliament of Victoria, except for certain 'entrenched' provisions that require either an absolute majority in both houses, a three-fifths majority in both houses, or the approval of the Victorian people in a referendum, depending on the provision."

q = 'When did Victoria enact its constitution?'

answer = model.predict(doc,q)

print(answer['answer'])
# 1975
print(answer.keys())
# dict_keys(['answer', 'start', 'end', 'confidence', 'document']))

In [8]:
!python kg-extraction.py --input_txt zelda.txt

2023-06-18 19:06:56.922567: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-18 19:06:57.091906: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-18 19:06:57.983854: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-06-18 19:06:57.983950

In [ ]:
#!pip uninstall -y transformers
#!pip install transformers==3.5